In [18]:
# Install required packages
!pip install -q pymupdf scikit-learn gtts indic-nlp-library sentence-transformers

In [19]:
# Import libraries
import fitz  # PyMuPDF
import re
import os
from gtts import gTTS
from IPython.display import Audio, display, HTML
from indicnlp import loader
from indicnlp.tokenize import sentence_tokenize
from sentence_transformers import SentenceTransformer, util

In [20]:
# Setup Indic NLP
INDIC_RESOURCES_PATH = "/content/indic_nlp_resources"
if not os.path.exists(INDIC_RESOURCES_PATH):
    !git clone https://github.com/anoopkunchukuttan/indic_nlp_resources.git {INDIC_RESOURCES_PATH}
os.environ['INDIC_RESOURCES_PATH'] = INDIC_RESOURCES_PATH
loader.load()

In [21]:
# Helper functions
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    return "".join([page.get_text() for page in doc])

def preprocess_text(text):
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^\u0900-\u097F\s\.,\-]', '', text)
    return text.strip()

def split_sentences(text, lang='mr'):
    return sentence_tokenize.sentence_split(text, lang)

def answer_query(query):
    query = preprocess_text(query)
    if not query:
        return "कृपया वैध विचार करा."
    query_embedding = model.encode(query, convert_to_tensor=True)
    cos_scores = util.pytorch_cos_sim(query_embedding, sentence_embeddings)[0]
    best_idx = cos_scores.argmax().item()
    if cos_scores[best_idx] < 0.4:
        return "माहिती सापडली नाही."
    response = sentences[best_idx]
    response = re.sub(r'अ\. 􀀗 \.', '', response)
    response = re.sub(r'\s+', ' ', response).strip()
    return response

def speak_marathi(text):
    # Convert text to speech (TTS)
    tts = gTTS(text=text, lang='mr')
    filename = "/content/response.mp3"
    tts.save(filename)

    # Provide a downloadable link for the MP3 file
    display(HTML(f'<a href="{filename}" download>Download the audio response</a>'))

    # Attempt to play the audio in the notebook (this may work in some environments)
    display(Audio(filename, autoplay=True))

In [23]:
# Load and process the PDF
pdf_path = "/content/Extra-Ordinary_CENTRAL-SECTION_Part-4-Marathi-.pdf"
print("Extracting text...")
raw_text = extract_text_from_pdf(pdf_path)
clean_text = preprocess_text(raw_text)
sentences = split_sentences(clean_text)
print(f"Total sentences: {len(sentences)}")

Extracting text...
Total sentences: 20


In [24]:
# Load sentence transformer
print("Encoding sentences...")
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
sentence_embeddings = model.encode(sentences, convert_to_tensor=True)

Encoding sentences...


In [ ]:
# Interactive assistant with accuracy feedback
correct_count = 0
total_count = 0
print("\nमराठी व्हॉईस असिस्टंट तयार आहे! प्रश्न विचारा (बाहेर पडण्यासाठी 'exit' टाइप करा):")
while True:
    query = input("\nतुमचा प्रश्न: ")
    if query.strip().lower() == 'exit':
        print("कायमचा संपवला.")
        break
    response = answer_query(query)
    print("उत्तर:", response)
    speak_marathi(response)

# Optional accuracy check (for manual verification)
    expected = input("✅ योग्य उत्तर काय असायला हवं होतं? (Enter द्या वगळण्यासाठी): ").strip()
    total_count += 1
    if expected:
        expected_clean = preprocess_text(expected)
        response_clean = preprocess_text(response)
        if expected_clean in response_clean:
            print("बरोबर उत्तर दिलं.")
            correct_count += 1
        else:
            print("चुकीचं उत्तर.")
            print(f"अपेक्षित: {expected}")
    else:
        print("पडताळणी वगळली.")

    accuracy = (correct_count / total_count) * 100
    print(f"संपूर्ण अचूकता: {accuracy:.2f}% ({correct_count}/{total_count})")


मराठी व्हॉईस असिस्टंट तयार आहे! प्रश्न विचारा (बाहेर पडण्यासाठी 'exit' टाइप करा):

तुमचा प्रश्न: महाराष्ट्र अधिनियम काय आहे?
उत्तर: महाराष्ट्र शासन , विधि व न्याय विभाग .


✅ योग्य उत्तर काय असायला हवं होतं? (Enter द्या वगळण्यासाठी): महाराष्ट्र शासन , विधि व न्याय विभाग .
बरोबर उत्तर दिलं.
संपूर्ण अचूकता: 100.00% (1/1)

तुमचा प्रश्न: महाराष्ट्र अधिनियम २०२२ चा काय आहे?
उत्तर: पृष्ठे ५ , किंमत रुपये २३.०० अनुक्रमणिका सन २०२३ चा महाराष्ट्र अधिनियम क्रमांक ८. - महाराष्ट्र अधिसंख्य पदांची निर्मिती व निवड केलेल्या उमेदवारांची नियुक्ती अधिनियम , २०२२ यामध्ये सुधारणा करण्याकरिता अधिनियम .


✅ योग्य उत्तर काय असायला हवं होतं? (Enter द्या वगळण्यासाठी): शालेय शिक्षण विभाग काय आहे?
चुकीचं उत्तर.
अपेक्षित: शालेय शिक्षण विभाग काय आहे?
संपूर्ण अचूकता: 50.00% (1/2)

तुमचा प्रश्न: शालेय शिक्षण विभाग काय आहे?
उत्तर: गृह विभाग राज्य उत्पादन शुल्क शालेय शिक्षण विभाग ११ १२ वैद्यकीय शिक्षण व औषधिद्रव्ये विभाग एक अधीक्षक स्त्री दोन सहायक प्रकल्प अधिकारी सांख्यिकी अधिकारी प्रशासकीय अधिकारी संशोधन अधिकारी गृह प्रबंधक एक स्थापत्य अभियांत्रिकी सहायक दोन कनिष्ठ स्थापत्य अभियंता तीन कनिष्ठ लिपिक चार कनिष्ठ लिपिक - टंकलेखक पाच वाहनचालक सहा खानसामा सात मैलमजूर आठ चौकीदार नऊ शिपाई एक पोलीस उप अधीक्षक सहायक पोलीस आयुक्त दोन सहायक प्रादेशिक परिवहन अधिकारी तीन सहायक आयुक्त चार उप अधीक्षक पाच दुय्यम निरीक्षक एक उप शिक्षण अधिकारी एक वरिष्ठ लिपिक एकूण ४ क ब क क क ड ड ड ड अ ब क अ क ५ १ ४ २ १ २ ६ २ १२ २ १ ४ १ १५५३. .


✅ योग्य उत्तर काय असायला हवं होतं? (Enter द्या वगळण्यासाठी): शालेय शिक्षण विभागातील संपूर्ण माहिती
चुकीचं उत्तर.
अपेक्षित: शालेय शिक्षण विभागातील संपूर्ण माहिती
संपूर्ण अचूकता: 33.33% (1/3)
